In [1]:
from __future__ import print_function
import argparse
import os
import random

import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input,merge
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import *
from keras.layers.wrappers import TimeDistributed
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, UpSampling2D,Conv2DTranspose
from keras.layers.recurrent import LSTM
from keras.regularizers import *
from keras.layers.normalization import *
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
dataset='cifar10'
dataroot='f1'
workers=1
batchSize=5
imageSize=32
nz=400
ngf=64
ndf=64
niter=25
lr=0.0002
beta1=0.5
cuda=True
ngpu=1
netG=''
netD=''
outf='f1'
manualSeed=10
try:
    os.makedirs(outf)
except OSError:
    pass


In [3]:
import os
I='A/test/'
ls1=os.listdir(I)
M='B/test/'
ls2=os.listdir(M)

import scipy.misc as sm
import numpy as np
import scipy.io as si
names=[]
lls2=len(ls2)
m=int(0.9*lls2)

X=np.zeros((m,256,256,3))
Y=np.zeros((m,256,256,3))
Xtest=np.zeros((lls2-m,256,256,3))
Ytest=np.zeros((lls2-m,256,256,3))
count=0
for ind,i in enumerate(ls2):
    print(ind)
    if(ind<m):
        names.append(i)
        #temp=i.split('__')[0]
        tmp1=sm.imread(I+i)
        X[ind,:]=tmp1/255.0
        tmp1=sm.imread(M+i)
        Y[ind,:]=tmp1/255.0
    else:
        #break
        names.append(i)
        tmp1=sm.imread(I+i)
        Xtest[count,:]=tmp1/255.0
        tmp1=sm.imread(M+i)
        Ytest[count,:]=tmp1/255.0
        count=count+1
    
    
    
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [72]:
def larger_model():
    model = Sequential()
    model.add(Convolution2D(32,(4,4),strides=1,padding='same',input_shape=(256,256,3) ))
    model.add(LeakyReLU(0.3))
    model.add(Convolution2D(32*2,(4,4),strides=1,padding='same' ))
    model.add(LeakyReLU(0.3))
    model.add(Convolution2D(32*4,(4,4),strides=1,padding='same'))
    model.add(LeakyReLU(0.3))
    model.add(Convolution2D(32*4,(4,4),strides=1,padding='same' ))
    model.add(LeakyReLU(0.3))
    model.add(Convolution2D(3,(4,4),strides=1,padding='same'))
    model.add(LeakyReLU(0.3))
    return model


In [73]:
def discr():
    model = Sequential()
    model.add(Convolution2D(32,(4,4),activation='relu',strides=2,input_shape=(256,256,3) ))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(32*2,(4,4),activation='relu',strides=2 ))
    model.add(Convolution2D(32*4,(4,4),activation='relu',strides=2))
    model.add(Convolution2D(32*2,(4,4),activation='relu',strides=2))
    model.add(Convolution2D(1,(4,4),activation='sigmoid',strides=2))
    
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model


In [86]:
import keras.backend as K

epsilon = 1.0e-9
def custom_objective(yTrue, yPred):
    
    return K.sum(K.log(yTrue) - K.log(yPred))

In [87]:
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val
        
gen=larger_model()
dis=discr()
make_trainable(dis, False)

In [89]:
def mixed(x,y):
    model=Sequential()
    model.add(x)
    model.add(y)
    model.compile(loss=custom_objective,optimizer='adam')
    return model
    

In [90]:
GAN=mixed(gen,dis)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_27 (Sequential)   (None, 256, 256, 3)       434019    
_________________________________________________________________
sequential_28 (Sequential)   (None, 6, 6, 1)           297761    
Total params: 731,780
Trainable params: 434,019
Non-trainable params: 297,761
_________________________________________________________________


In [91]:
gen.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_85 (Conv2D)           (None, 256, 256, 32)      1568      
_________________________________________________________________
leaky_re_lu_52 (LeakyReLU)   (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 256, 256, 64)      32832     
_________________________________________________________________
leaky_re_lu_53 (LeakyReLU)   (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_87 (Conv2D)           (None, 256, 256, 128)     131200    
_________________________________________________________________
leaky_re_lu_54 (LeakyReLU)   (None, 256, 256, 128)     0         
_________________________________________________________________
conv2d_88 (Conv2D)           (None, 256, 256, 128)     262272    
__________

In [92]:
dis.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_90 (Conv2D)           (None, 127, 127, 32)      1568      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 129, 129, 32)      0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 63, 63, 64)        32832     
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 30, 30, 128)       131200    
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 14, 14, 64)        131136    
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 6, 6, 1)           1025      
Total params: 297,761
Trainable params: 0
Non-trainable params: 297,761
_________________________________________________________________


In [93]:
bs=20
make_trainable(dis,True)

for epoch in range(20):
    for i in range(100):
        ind=np.random.permutation(X.shape[0])[0:bs]
        data=X[(ind)]        
        data2=Y[(ind)]

        gen_im=gen.predict(data)
        train_D = np.concatenate((data2, gen_im))
        
        y_D=np.zeros((2*bs,6,6,1))
        y_D[0:bs,:]=1
        y_D[bs:,:]=0
        d_loss  = dis.train_on_batch(train_D,y_D)
        #print(gen_im.shape)

        y_G=np.zeros((bs,6,6,1))
        y_G[:,:]=1
        make_trainable(dis,False)
        g_loss=GAN.train_on_batch(data,y_G)
        make_trainable(dis,True)
        if(i%40==0):
            print((epoch,i,d_loss,g_loss))
        #break
    #break

(0, 0, 0.69073188, 498.23657)


KeyboardInterrupt: 

In [83]:
import scipy.misc as sm
#print(X[50:51,:])
temp=gen.predict(X[50:51,:])
#print(temp)
sm.imsave('vap.png',temp.reshape((256,256,3)))
sm.imsave('vap2.png',Y[50:51].reshape((256,256,3)))
